In [ ]:
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

from fbprophet import Prophet

# Initialising parameters
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['axes.grid'] = False

In [ ]:
!pip install pyspark # Install pyspark

In [ ]:
from pyspark.sql import SparkSession 
import pyspark

spark = SparkSession.builder.master('local').getOrCreate()
# Load csv file in pandas
df = pd.read_csv("/content/sample_data/26 counties monthly CPI.csv")

In [ ]:
print("Shape of the dataset: ", df.shape)
print("\nSize of the pandas table:", df.info())

In [ ]:
# OBSERVATION: Number of rows we have: 30,482 with 8 features     1.9+ MB

In [ ]:
df.head()

In [ ]:
len(df['LOCATION'].unique()), df['LOCATION'].unique()

In [ ]:
# Keeping only 4 countries 
df=df[df['LOCATION'].apply(lambda x: x=='IND'or x=='USA' or x=='RUS' or x=='FIN')]

In [ ]:
len(df['LOCATION'].unique()), df['LOCATION'].unique()

In [ ]:
print("Shape of the dataset: ", df.shape)
print("\nSize of the pandas table:", df.info())

In [ ]:
df.rename(columns={'TIME': 'Date', 'LOCATION': 'Location'}, inplace=True)

In [ ]:
df=df[['Date', 'Value', 'Location']]

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
df['Value']=df['Value'].astype(float)
df.head()

In [ ]:
df['Date'].min(), df['Date'].max()

In [ ]:
item_df = df.set_index('Date')
item_df=item_df[['Value', 'Location']]
item_df.head()

In [ ]:
item_df[item_df['Location'] == 'FIN']['Value'].plot()

# Stationarity Check

In [ ]:
for Location in list(df['Location'].unique()):
  item_df[item_df['Location'] == Location]['Value'].plot(title=Location)
  plt.show()

In [ ]:
#zip basically combines result,labels
# Augmented Dickey–Fuller test (ADF) tests
from statsmodels.tsa.stattools import adfuller
def adfuller_test(sales):
    result=adfuller(sales)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")
        
# source: https://www.kaggle.com/code/avi111297/predicting-sales-using-arima-sarimax-tsf-model

In [ ]:
# Augmented Dickey–Fuller test (ADF) tests
for Location in list(df['Location'].unique()):
  print("\nLocation is: ", Location)
  adfuller_test(item_df[item_df['Location'] == Location]['Value'])

In [ ]:
!pip install statsmodels

In [ ]:
df.set_index("Date")[['Value']].head()

In [ ]:
dictCrypto={'AUT': 1, 'BEL': 2, 'CAN': 3, 'CZE': 4, 'DNK': 5, 'FIN': 6, 'FRA': 7, 'DEU': 8, 'GRC': 9,
        'HUN': 10, 'ISL': 11, 'IRL': 12, 'ITA': 13, 'JPN': 14, 'KOR': 15, 'LUX': 16, 'MEX': 17, 'NLD': 18,
        'NOR': 19, 'POL': 20, 'PRT': 21, 'SVK': 22, 'ESP': 23, 'SWE': 24, 'CHE': 25, 'TUR': 26, 'GBR': 27,
        'USA': 28, 'BRA': 29, 'CHL': 30, 'CHN': 31, 'EST': 32, 'IND': 33, 'IDN': 34, 'ISR': 35, 'RUS': 36,
        'SVN': 37, 'ZAF': 38, 'OECD': 39, 'OECDE': 40, 'G-7': 41, 'COL': 42, 'LVA': 43, 'SAU': 44, 'EA19': 45,
        'ARG': 46, 'LTU': 47, 'G-20': 48, 'EU27_2020': 49}
dictCrypto = {'FIN': 1, 'USA': 2, 'IND': 3, 'RUS': 4} 
df['LocId']=df['Location'].apply(lambda x: dictCrypto[x])

In [ ]:
df.head()

In [ ]:
import time 
from sklearn.metrics import mean_absolute_error
def forecast_sales(crypto_pd):
  model = Prophet(interval_width=0.95, seasonality_mode= 'multiplicative', daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
  model.fit(crypto_pd)
  future_pd = model.make_future_dataframe(periods=30, freq='M')
  forecast_pd = model.predict(future_pd)
  f_pd = forecast_pd[['ds', 'yhat', 'yhat_upper', 'yhat_lower']].set_index('ds')
  st_pd = crypto_pd[['ds', 'Location', 'y']].set_index('ds')
  result_pd = f_pd.join(st_pd, how='left')
  result_pd.reset_index(level=0, inplace=True)
  result_pd['Location'] = crypto_pd['Location'].iloc[0]

  #from fbprophet.diagnostics import cross_validation
  #cv_results = cross_validation( model = model, initial = pd.to_timedelta(5,unit="d"), horizon = pd.to_timedelta(5,unit="d"))
  #print("cv_results:", cv_results)
  
  return result_pd[['ds', 'Location', 'y', 'yhat', 'yhat_upper', 'yhat_lower']]


tick= time.time()
for cryptoName in list(df['Location'].unique()):
  pdIndividualCrypto=df[df['Location'] == Location][['Date','Location', 'Value']].rename(columns={'Date': 'ds', 'Value': 'y'})
  final_df=forecast_sales(pdIndividualCrypto)
  

  # calculate Mean Absolute Error (MAE) between expected and predicted values for december
  y_true = final_df.dropna()['y'].values
  y_pred = final_df.dropna()['yhat'].values

  mae = mean_absolute_error(y_true, y_pred)
  print(Location, ': MAE: %.3f' % mae)

  final_df[['y', 'yhat']].plot(title=Location + ': MAE: %.3f' % mae)
  
tock=time.time()
TotalTime=tock-tick
print("Total time taken: {} sec.s".format(round(tock-tick, 3)))

## Spark 

In [ ]:
sdf = spark.createDataFrame(df)
sdf.printSchema() #data type of each col
sdf.show(5) #It gives you head of pandas DataFrame
sdf.count() #500 records

In [ ]:
sdf.select(['LocId']).groupby('LocId').agg({'LocId': 'count'}).show()

In [ ]:
sdf.createOrReplaceTempView("Crypto")
spark.sql("select LocId, count(*) from Crypto group by LocId order by LocId").show()

In [ ]:
sql = "SELECT LocId, Date as ds, sum(Value) as y FROM Crypto GROUP BY LocId, ds ORDER BY LocId, ds"
spark.sql(sql).show()

In [ ]:
store_part = (spark.sql(sql).repartition(spark.sparkContext.defaultParallelism, ['LocId'])).cache()
sdf.explain()

In [ ]:
from pyspark.sql.types import *
result_schema = StructType([
                  StructField('ds', TimestampType()),
                  StructField('LocId', IntegerType()),
                  StructField('y', DoubleType()),
                  StructField('yhat', DoubleType()),
                  StructField('yhat_upper', DoubleType()),
                  StructField('yhat_lower', DoubleType())
])

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP)
def forecast_sales(crypto_pd):
  #model = Prophet(interval_width=0.95, seasonality_mode= 'multiplicative', weekly_seasonality=True, yearly_seasonality=True)
  model = Prophet(interval_width=0.95, seasonality_mode= 'multiplicative', daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
  model.fit(crypto_pd)
  future_pd = model.make_future_dataframe(periods=30, freq='M')
  forecast_pd = model.predict(future_pd)
  f_pd = forecast_pd[['ds', 'yhat', 'yhat_upper', 'yhat_lower']].set_index('ds')
  st_pd = crypto_pd[['ds', 'LocId', 'y']].set_index('ds')
  result_pd = f_pd.join(st_pd, how='left')
  result_pd.reset_index(level=0, inplace=True)
  result_pd['LocId'] = crypto_pd['LocId'].iloc[0]

  return result_pd[['ds', 'LocId', 'y', 'yhat', 'yhat_upper', 'yhat_lower']]

In [ ]:
from pyspark.sql.functions import current_date
tick=time.time()
results = (store_part.groupby('LocId').apply(forecast_sales).withColumn('training_date', current_date()))
results.cache()
results.show(5)
tock=time.time()
print("Total time taken: {} seconds".format((tock-tick)/60))

In [ ]:
results.coalesce(1)
print(results.count())
results.createOrReplaceTempView('forecasted')
spark.sql("SELECT LocId, count(*) FROM  forecasted GROUP BY LocId").show()

In [ ]:
final_df = results.toPandas()
final_df.head()

In [ ]:
dictCrypto={1: 'bitcoin', 2: 'ethereum', 3: 'cardano', 4: 'tether'}
dictCrypto={1: 'AUT', 2:'BEL', 3:'CAN', 4:'CZE', 5:'DNK', 6:'FIN', 7:'FRA', 8:'DEU',
            9: 'GRC', 10: 'HUN', 11: 'ISL', 12: 'IRL', 13: 'ITA', 14: 'JPN', 15: 'KOR',
            16: 'LUX', 17: 'MEX', 18: 'NLD', 19: 'NOR', 20: 'POL', 21: 'PRT', 22:'SVK', 
            23:'ESP', 24:'SWE', 25:'CHE', 26:'TUR', 27:'GBR', 28: 'USA', 29: 'BRA', 30: 'CHL', 31: 'CHN', 
            32: 'EST', 33: 'IND', 34: 'IDN', 35: 'ISR', 36: 'RUS', 37: 'SVN', 38: 'ZAF', 
            39: 'OECD', 40: 'OECDE', 41: 'G-7', 42:'COL', 43:'LVA', 44:'SAU', 45:'EA19',
            46: 'ARG', 47:'LTU', 48:'G-20', 49:'EU27_2020'}
dictCrypto = {1: 'FIN', 2: 'USA', 3: 'IND', 4: 'RUS'} 

final_df['Location']=final_df['LocId'].apply(lambda x: dictCrypto[x])
final_df.head()

In [ ]:
final_df[final_df['Location'] == 'FIN'][['y', 'yhat']].plot()

In [ ]:
final_df = final_df.set_index('ds')
for Location in list(final_df.Location.unique()):
  #final_df.query('CryptoName == {}'.format(CryptoName))[['y', 'yhat']].plot()
  
  pdTemp=final_df[final_df['Location'] == Location][['y', 'yhat']]
  
  # calculate Mean Absolute Error (MAE) between expected and predicted values for december
  y_true = pdTemp.dropna()['y'].values
  y_pred = pdTemp.dropna()['yhat'].values

  mae = mean_absolute_error(y_true, y_pred)
  print(Location, ': MAE: %.3f' % mae)

  pdTemp[['y', 'yhat']].plot(title=Location + ': MAE: %.3f' % mae)
  plt.show()